In [1]:
import os
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon
from shapely.geometry import Point

In [2]:
directory = "data/tables/pagexml"
image_name = "NL-HaNA_2.10.50_45_0110.jpg"

In [3]:
# Go outside the src directory
os.chdir("..")
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /Users/sarah_shoilee/codeProjects/stamboekn_KE


need to construct HTML Table form transkribus XML

In [4]:
from src.utils import pagexml_to_html

pagexml_file = os.path.join(directory, image_name+ ".xml")
output_file = os.path.join("data/tables/html", image_name + ".html")
pagexml_to_html(pagexml_file, output_file)

"<table border='1'>\n  <tr>\n    <td id='t1c1' row='0' col='0' colspan='1' rowspan='1'>Namen en Toenamen.</td>\n    <td id='t1c2' row='0' col='1' colspan='1' rowspan='1'></td>\n    <td id='t1c3' row='0' col='2' colspan='1' rowspan='1'>Namen der Ouders, datum van Geboorte,<br/>Geboorteplaats en laatste Woonplaats.</td>\n    <td id='t1c4' row='0' col='3' colspan='1' rowspan='1'>g naar ande<br/>braden, van o<br/>Pensiden.</td>\n    <td id='t1c5' row='0' col='4' colspan='1' rowspan='1'></td>\n    <td id='t1c6' row='0' col='5' colspan='1' rowspan='1'>Camns van aetesen in ondterte / hn ietere mnad / bedere vestosten, hetonere vonden<br/>uitstekende daden of bijzondere<br/>verrigtingen.</td>\n  </tr>\n  <tr>\n    <td id='t1c7' row='1' col='0' colspan='1' rowspan='1'>Behm<br/>Geen</td>\n    <td id='t1c8' row='1' col='1' colspan='1' rowspan='1'>Pie</td>\n    <td id='t1c9' row='1' col='2' colspan='1' rowspan='1'>Vader Jan Nicolaas<br/>Weeder Maenweelke Eller<br/>Seberen den 1 Junij 175<br/>Gebar

### Measure mAP

In [6]:
from src.metrics import compute_mAP

gt_file = "data/labels/polygons/NL-HaNA_2.10.50_45_0091.jpg.polygons.json"
pred_file = "data/tables/pagexml/NL-HaNA_2.10.50_45_0091.jpg.xml"
mAP = compute_mAP(gt_file, pred_file)


Final Mean Average Precision (mAP): 0.9471


### Measure TED

In [7]:
from src.utils import format_td
from src.metrics import TEDS
def calculate_TEDS(ground_truth_html, predicted_html):
    # predicted_html = format_td(predicted_html)
    ground_truth_html = format_td(ground_truth_html)

    teds = TEDS(structure_only=False)
    teds_score = teds.evaluate(ground_truth_html, predicted_html)

    teds_struct = TEDS(structure_only=True)
    teds_struct_score = teds_struct.evaluate(ground_truth_html, predicted_html)
    
    print(f"TEDS: {teds_score:.4f}")
    print(f"TEDS-Struct: {teds_struct_score:.4f}")

    return teds_score, teds_struct_score

In [9]:
import re
with open(os.path.join("data/tables/html/", image_name+ '.html'), 'r', encoding='utf-8') as f:
    constructed_html = f.read()

with open(os.path.join("data/labels/tables", image_name+ '.html'), 'r', encoding='utf-8') as f:
    label_html = f.read()

calculate_TEDS(label_html, constructed_html)

TEDS: 0.9146
TEDS-Struct: 0.9840


(0.9145809422119834, 0.984)

### Information Extraction

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(constructed_html, 'html.parser')

rows = soup.find_all('tr')

# --- Step 1: make a grid keeping track of rowspans ---
logical_rows = []
rowspans = {}  # {(col_idx): [remaining_rows, cell_content]}

for r_idx, tr in enumerate(rows):
    # Start with any carried over rowspans from previous rows
    current_row = []
    to_remove = []

    # Fill carried-down cells first
    for col_idx, (remaining, cell) in rowspans.items():
        current_row.append(cell)
        rowspans[col_idx][0] -= 1
        if rowspans[col_idx][0] <= 0:
            to_remove.append(col_idx)
    for col_idx in to_remove:
        del rowspans[col_idx]

    # Now add new cells from this row
    c_idx = 0
    for td in tr.find_all('td'):
        while any(k == c_idx for k in rowspans):  # skip columns occupied by carried cells
            c_idx += 1
        text = td.get_text(" ", strip=True)
        cell_id = td.get('id')
        r_idx = int(td.get('row', r_idx))
        c_idx = int(td.get('col', c_idx))
        rowspan = int(td.get('rowspan', 1))
        colspan = int(td.get('colspan', 1))

        cell_data = {
            'text': text,
            'id': cell_id,
            'row': r_idx,
            'col': c_idx,
            'rowspan': rowspan,
            'colspan': colspan
        }

        # Place cell in current row
        current_row.append(cell_data)

        # Store for future rows if rowspan > 1
        if rowspan > 1:
            rowspans[c_idx] = [rowspan - 1, cell_data]

        c_idx += colspan

    logical_rows.append(current_row)


In [11]:
import re
import json
from src.person_info_extraction import extract_info_regex, extract_info_LLM

persons = []

for i, row in enumerate(logical_rows):
    person = {}
        
    # print(generate_prompt(cells))
    person = json.loads(extract_info_LLM(row))
    if all(v['value']==None for v in person.values()):
        continue

    if person:
        persons.append(person)

unique_persons = {json.dumps(person, sort_keys=True) for person in persons}
unique_persons_list = [json.loads(p) for p in unique_persons]
        
json_obj = {"persons": unique_persons_list}
print(json.dumps(json_obj, indent=2, ensure_ascii=False))

{
  "persons": [
    {
      "geboorte_datum": {
        "cell": "t1c9",
        "value": "1 Junij 175"
      },
      "geboorte_plaats": {
        "cell": "t1c9",
        "value": "Levenberg"
      },
      "laatste_woonplaats": {
        "cell": null,
        "value": null
      },
      "moeder": {
        "cell": null,
        "value": null
      },
      "vader": {
        "cell": "t1c9",
        "value": "Jan Nicolaas Weeder Maenweelke Eller Seberen"
      }
    },
    {
      "geboorte_datum": {
        "cell": "t1c15",
        "value": "4 Juny 1796"
      },
      "geboorte_plaats": {
        "cell": "t1c15",
        "value": "Senwigk"
      },
      "laatste_woonplaats": {
        "cell": "t1c15",
        "value": "Eijden"
      },
      "moeder": {
        "cell": "t1c15",
        "value": "Melunina Caroline Sophiar geb: Drechler"
      },
      "vader": {
        "cell": null,
        "value": null
      }
    }
  ]
}


In [12]:
with open(f"data/json/{image_name}.json", "w", encoding='utf-8') as json_file:
    json.dump(json_obj, json_file, ensure_ascii=False, indent=2)#!/usr/bin/env python3

In [13]:
from src.metrics import best_match_similarity
# from src.metrics import calculate_normalized_information_distance

with open(f"data/json/{image_name}.json", 'r', encoding='utf-8') as f:
    constructed_html = json.load(f)
with open(os.path.join("data/labels/info", image_name.replace('.jpg', '.json')), 'r', encoding='utf-8') as f:
    label_html = json.load(f)

# calculate_normalized_information_distance(constructed_html, label_html)
print(best_match_similarity(label_html.get("persons", []), constructed_html.get("persons", [])))


0.3079254079254079


In [14]:
from src.metrics import infomration_extraction_precision_recall

with open(f"data/json/{image_name}.json", 'r', encoding='utf-8') as f:
    constructed_html = json.load(f)
with open(os.path.join("data/labels/info", image_name.replace('.jpg', '.json')), 'r', encoding='utf-8') as f:
    label_html = json.load(f)

# calculate_normalized_information_distance(constructed_html, label_html)
print(infomration_extraction_precision_recall(label_html.get("persons", []), constructed_html.get("persons", []), threshold=0.4))


(0.2, 0.3)


### KG Construction

In [ ]:
# load json from file
with open(f"data/json/{image_name}.json", "r", encoding="utf-8") as f:
    json_obj = json.load(f)


In [ ]:
# CONSTRUCT ASSERSION TRIPLES
from rdflib import Graph, ConjunctiveGraph, Namespace, URIRef, Literal, RDF

FOAF = Namespace("http://xmlns.com/foaf/0.1/")
EX = Namespace("http://example.org/ontology/")
PROV = Namespace("http://www.w3.org/ns/prov#")

cg = ConjunctiveGraph()
cg.bind("foaf", FOAF)
cg.bind("ex", EX)
cg.bind("prov", PROV)

# Mapping from json keys to RDF predicates
predicate_map = {
    "vader": EX.vader,
    "moeder": EX.moeder,
    "geboorte_datum": EX.geboorteDatum,
    "geboorte_plaats": EX.geboortePlaats,
    "laatste_woonplaats": EX.laatsteWoonplaats
}

for idx, person in enumerate(json_obj["persons"], start=1):
    person_uri = URIRef(f"http://example.org/person/{idx}")
    assertion_graph_uri = URIRef("http://example.org/assertion")
    assertion_graph = Graph(store=cg.store, identifier=assertion_graph_uri)
    assertion_graph.add((person_uri, RDF.type, FOAF.Person))

    provenance_graph_uri = URIRef("http://example.org/provenance")
    provenance_graph = Graph(store=cg.store, identifier=provenance_graph_uri)

    for key, value_dict in person.items():
        value = value_dict["value"]
        cell_id = value_dict["cell"]
        predicate = predicate_map.get(key)
        if predicate:
            # Named graph for each cell
            graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            ng = Graph(store=cg.store, identifier=graph_uri)
            ng.add((person_uri, predicate, Literal(value)))
            

cg.serialize(f"data/triples/{image_name}_assersion.trig", format='trig')


In [ ]:
end_time = "2025-09-01T12:00:00Z"  # Example end time, replace with actual time if needed
start_time = "2025-09-01T10:00:00Z"  # Example start time, replace with actual time if needed

# CONSTRUCT PROVENANCE TRIPLES
from lxml import etree

def add_provenance_graph(pagexml_path, stamboek_nummer=image_name):
    tree = etree.parse(pagexml_path)
    root = tree.getroot()

    EX = Namespace("http://example.org/ontology/")
    IMG = Namespace("http://example.org/image_ontology/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")  
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    PROV = Namespace("http://www.w3.org/ns/prov#")
    CSVW = Namespace("http://www.w3.org/ns/csvw#")

    # Create RDF graph
    g = Graph()
    g.bind("ex", EX)
    g.bind("img", IMG)
    g.bind("rdf", RDF)
    g.bind("rdfs", RDFS)
    g.bind("prov", PROV)
    g.csvw = ("csvw", CSVW)


    # Find TableRegion(s)
    table_regions = root.findall(".//{*}TableRegion")

    for table_region in table_regions:
        for cell in table_region.findall(".//{*}TableCell"):
            cell_id = cell.get('id')
            rows = cell.get('row')
            cols = cell.get('col')
            Coords = cell.find(".//{*}Coords")
            coords_points = Coords.get('points') if Coords is not None else None
            
            # cell uri
            named_graph_uri = URIRef(f"http://example.org/graph/{cell_id}")
            cell_uri = URIRef(f"http://example.org/id/{cell_id}")
            provenance_graph.add((named_graph_uri, PROV.wasDerivedFrom, cell_uri))
            g.add((cell_uri, RDF.type, PROV.Entity))
            g.add((cell_uri, RDFS.label, Literal(f"Cell {cell_id} from {stamboek_nummer}")))
            
            g.add((cell_uri, RDF.type, CSVW.Cell))
            g.add((cell_uri, CSVW.rowNumber, Literal(rows)))
            g.add((cell_uri, CSVW.columnNumber, Literal(cols)))
            g.add((cell_uri, EX.ImageRegion, Literal(coords_points)))

            # agents
            agent_1 = URIRef("http://example.org/agent/1")
            g.add((agent_1, RDF.type, PROV.Agent))
            g.add((agent_1, RDFS.label, Literal("Sarah Shoilee")))
            g.add((named_graph_uri, PROV.wasAttributedTo, agent_1))
            project_agent = URIRef("http://example.org/agent/2")
            g.add((project_agent, RDF.type, PROV.Agent))
            g.add((project_agent, RDFS.label, Literal("Pressing Matter Project")))
            g.add((agent_1, PROV.actedOnBehalfOf, project_agent))

            # activity
            stamboekenKGConstructionactivity = URIRef(f"http://example.org/activity/stamboekenKGConstructionactivity/{cell_id}")
            tableConstructionactivity = URIRef(f"http://example.org/activity/TableExtraction/{cell_id}")
            informationExtractionactivity = URIRef(f"http://example.org/activity/InformationExtraction/{cell_id}")
            KGConstructionactivity = URIRef(f"http://example.org/activity/KGConstruction/{cell_id}")
            
            g.add((stamboekenKGConstructionactivity, RDF.type, PROV.Activity))
            g.add((named_graph_uri, PROV.wasGeneratedBy, stamboekenKGConstructionactivity))
            g.add((stamboekenKGConstructionactivity, PROV.wasAssociatedWith, agent_1))
            g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, tableConstructionactivity))
            g.add((tableConstructionactivity, RDF.type, PROV.Activity))
            g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, informationExtractionactivity))
            g.add((informationExtractionactivity, RDF.type, PROV.Activity))
            g.add((informationExtractionactivity, PROV.used, cell_uri))
            g.add((stamboekenKGConstructionactivity, PROV.wasInformedBy, KGConstructionactivity))
            g.add((KGConstructionactivity, RDF.type, PROV.Activity))
            g.add((KGConstructionactivity, PROV.used, cell_uri))

            g.add((stamboekenKGConstructionactivity,PROV.endedAtTime, Literal(end_time)))
            g.add((stamboekenKGConstructionactivity,PROV.startedAtTime, Literal(start_time)))

            # Create a Table instance URI
            table_uri = URIRef(f"http://example.org/Table/{cell_id}")
            g.add((table_uri, RDF.type, PROV.Entity))
            g.add((table_uri, RDF.type, CSVW.Table))
            g.add((table_uri, PROV.wasGeneratedBy, tableConstructionactivity))
            g.add((cell_uri, PROV.wasDerivedFrom, table_uri))
            
            # stamboeken
            stamboek_uri = URIRef(f"http://example.org/stamboek/{stamboek_nummer}")
            g.add((stamboek_uri, RDF.type, PROV.Entity))
            g.add((tableConstructionactivity, PROV.used, stamboek_uri))
            g.add((table_uri, PROV.wasDerivedFrom, stamboek_uri))
            national_archives = URIRef("http://example.org/agent/3")
            g.add((national_archives, RDF.type, PROV.Agent))
            g.add((national_archives, RDFS.label, Literal("Nationaal Archief")))
            g.add((stamboek_uri, PROV.wasAttributedTo, national_archives))
    
    g.serialize(f"data/triples/{image_name}_provenance.ttl", format='ttl')
add_provenance_graph(f"data/labels/{image_name}.xml")